In [3]:
import warnings
warnings.filterwarnings("ignore")

import json
import pprint
import numpy as np
import six
import tempfile
import tensorflow.compat.v2 as tf
# V2 Behavior is necessary to use TF2 APIs before TF2 is default TF version internally.
tf.enable_v2_behavior()
from tensorflow.keras.optimizers import *

from qkeras.autoqkeras import *
from qkeras import *
from qkeras.utils import model_quantize
from qkeras.qtools import run_qtools
from qkeras.qtools import settings as qtools_settings

from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds

print("using tensorflow", tf.__version__)

2022-05-07 01:14:45.956568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-07 01:14:45.956723: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


using tensorflow 2.5.0


In [4]:
from tensorflow.keras.initializers import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *

In [14]:
quantization_config = {
        "kernel": {
                "binary": 1,
                "stochastic_binary": 1,
                "ternary": 2,
                "stochastic_ternary": 2,
                "quantized_bits(2,1,1,alpha=1.0)": 2,
                "quantized_bits(4,0,1,alpha=1.0)": 4,
                "quantized_bits(8,0,1,alpha=1.0)": 8,
                "quantized_bits(16,0,1,alpha=1.0)": 16,
                "quantized_po2(4,1)": 4
        },
        "bias": {
                "quantized_bits(4,0,1)": 4,
                "quantized_bits(8,3,1)": 8,
                "quantized_bits(16,0,1,alpha=1.0)": 16,
                "quantized_po2(4,8)": 4
        },
        "activation": {
                "binary": 1,
                "ternary": 2,
                "quantized_relu_po2(4,4)": 4,
                "quantized_relu(3,1)": 3,
                "quantized_relu(4,2)": 4,
                "quantized_relu(8,2)": 8,
                "quantized_relu(8,4)": 8,
                "quantized_relu(16,8)": 16
        },
        "linear": {
                "binary": 1,
                "ternary": 2,
                "quantized_bits(4,1)": 4,
                "quantized_bits(8,2)": 8,
                "quantized_bits(16,10)": 16
        }
}

In [15]:
limit = {
    "Dense": [8, 8, 4],
    "Conv2D": [4, 8, 4],
    "DepthwiseConv2D": [4, 8, 4],
    "Activation": [4],
    "BatchNormalization": []
}

In [ ]:
goal = {
          "type": "bits",
          "params": {
              "delta_p": 8.0,
              "delta_n": 8.0,
              "rate": 2.0,
              "stress": 1.0,
              "input_bits": 8,
              "output_bits": 8,
              "ref_bits": 8,
              "config": {
                  "default": ["parameters", "activations"]
              }
          }
      }

In [5]:
x_test_em_barrel = np.load('x_test_em_barrel.npy')
x_train_em_barrel = np.load('x_train_em_barrel.npy')
y_train_targets = np.load('y_train_targets.npy')
y_test_targets = np.load('y_test_targets.npy')

In [7]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.regularizers import l1
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU

In [8]:
input1 = Input(shape=(56, 11, 4), name='em_barrel')
x1 = UpSampling2D(size=(1, 5), interpolation='nearest',
                    data_format='channels_last', name='up_sampling')(input1)
x2 = BatchNormalization(name='batch_norm',epsilon=25)(x1)
x2 = Conv2D(16, kernel_size=(5,5), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv', padding='same')(x2) 
x3 = BatchNormalization(name='batch_norm_1')(x2)
x4 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu')(x3)
x5 = MaxPooling2D((2,2), name='max_pooling2D')(x4)
x6 = Conv2D(32, kernel_size=(3,3), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_1', padding='same')(x5) 
x6 = BatchNormalization(name='batch_norm_2')(x6)
x7 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_1')(x6)
x8 = Conv2D(32, kernel_size=(3,3), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_2', padding='same')(x7) 
x8 = BatchNormalization(name='batch_norm_3')(x8)
x9 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_2')(x8)
x10 = MaxPooling2D((2,2), name='max_pooling2D_1')(x9)
x11 = Conv2D(64, kernel_size=(3,3), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_3', padding='same')(x10)
x11 = BatchNormalization(name='batch_norm_4')(x11)
x12 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_3')(x11)
x13 = Conv2D(64, kernel_size=(3,3), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_4', padding='same')(x12) 
x13 = BatchNormalization(name='batch_norm_5')(x13)
x14 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_4')(x13)
x15 = MaxPooling2D((2,2), name='max_pooling2D_2')(x14)
x16 = Conv2D(128, kernel_size=(3,3), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_5', padding='same')(x15) 
x16 = BatchNormalization(name='batch_norm_6')(x16)
x17 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_5')(x16)
x18 = Conv2D(128, kernel_size=(3,3), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_6', padding='same')(x17) 
x18 = BatchNormalization(name='batch_norm_7')(x18)
x19 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_6')(x18)
x20 = MaxPooling2D((2,2), name='max_pooling2D_3')(x19)
x21 = Conv2D(256, kernel_size=(3,3), strides=(1,1), kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=False,
               name='conv_7', padding='same')(x20) 
x21 = BatchNormalization(name='batch_norm_8')(x21)
x22 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_7')(x21)
x23 = QConv2DBatchnorm(256, kernel_size=(3,3), strides=(1,1),
                         kernel_quantizer="quantized_bits(16,4,alpha=1)", 
                         bias_quantizer="quantized_bits(16,4,alpha=1)",
                         kernel_initializer='lecun_uniform', 
                         kernel_regularizer=l1(0.0001), 
                         use_bias=True,
                         padding='same',
                         name='conv2d_batchnorm_8')(x22)
x24 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_8')(x23)
x25 = Flatten(name='flatten')(x24)
x26 = Dense(256, name='dense', #kernel_quantizer=quantized_bits(32,16,alpha='auto_po2'),
                               kernel_initializer='lecun_uniform',
                               kernel_regularizer=l1(0.0001),
                               use_bias=False)(x25)
x27 = BatchNormalization(name='batch_norm_10')(x26)
x28 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_9')(x27)
x29 = Dense(256, name='dense_1', #kernel_quantizer=quantized_bits(32,16,alpha='auto_po2'),
                               kernel_initializer='lecun_uniform',
                               kernel_regularizer=l1(0.0001),
                               use_bias=False)(x28)
x30 = BatchNormalization(name='batch_norm_11')(x29)
x31 = LeakyReLU(alpha = 0.30000001192092896, name='leaky_relu_10')(x30)
x32 = Dense(1, name='dense_2', #kernel_quantizer=quantized_bits(32,16,alpha='auto_po2'),
                               #bias_quantizer=quantized_bits(32,16,alpha='auto_po2'),
                               kernel_initializer='lecun_uniform',
                               kernel_regularizer=l1(0.0001),
                               use_bias=True)(x31)
output1 = Activation(name='activate', activation='relu')(x32)


model = Model(inputs= [input1], outputs=[output1], name='model')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
em_barrel (InputLayer)       [(None, 56, 11, 4)]       0         
_________________________________________________________________
up_sampling (UpSampling2D)   (None, 56, 55, 4)         0         
_________________________________________________________________
batch_norm (BatchNormalizati (None, 56, 55, 4)         16        
_________________________________________________________________
conv (Conv2D)                (None, 56, 55, 16)        1600      
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 56, 55, 16)        64        
_________________________________________________________________
leaky_relu (LeakyReLU)       (None, 56, 55, 16)        0         
_________________________________________________________________
max_pooling2D (MaxPooling2D) (None, 28, 27, 16)        0     

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from tensorflow.keras.layers import Activation
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm
from tensorflow.keras.callbacks import TensorBoard

In [11]:
train = True
if train:
    nadam = Nadam(lr=0.001)
    adam = Adam(learning_rate=3E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)
    model.compile(optimizer=adam, loss=['MeanAbsoluteError'], metrics=['mae'])
    callbacks = all_callbacks(stop_patience = 1000,
                              lr_factor = 0.5,
                              lr_patience = 10,
                              lr_epsilon = 0.000001,
                              lr_cooldown = 2,
                              lr_minimum = 0.0000001,
                              outputDir = 'keras_model_test')
    #callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=0),
    #              TensorBoard(log_dir="model_5/logs", profile_batch = 100000000)]
    #clbks = TensorBoard(log_dir="keras_model_v6/logs", profile_batch = 100000000)
    #callbacks.callbacks.append(clbks)
    #callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
    model.fit(x_train_em_barrel, y_train_targets, batch_size=1024,
              epochs=300, validation_split=0.1, shuffle=True,
              callbacks = callbacks.callbacks)
    # Save the model again but with the pruning 'stripped' to use the regular layer types
    #model = strip_pruning(model)
    model.save('keras_model_test/KERAS_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects
    co = {}
    _add_supported_quantized_objects(co)
    model = load_model('model_4/KERAS_check_best_model.h5', custom_objects=co)

Epoch 1/300


2022-05-07 01:16:26.967948: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-05-07 01:16:26.968093: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-05-07 01:16:26.968245: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-05-07 01:16:27.077246: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-07 01:16:27.077911: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3194080000 Hz


1/9 [==>...........................] - ETA: 23s - loss: 86.4410 - mae: 82.2382

2022-05-07 01:16:30.136544: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-05-07 01:16:30.136582: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


2/9 [=====>........................] - ETA: 11s - loss: 85.0273 - mae: 80.8211

...

9/9 [==============================] - 15s 2s/step - loss: 17.5989 - mae: 15.6358 - val_loss: 17.1222 - val_mae: 15.1592

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00228: val_loss did not improve from 17.08047

Epoch 00228: val_loss did not improve from 17.08047

Epoch 00228: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00228: saving model to keras_model_test/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 229/300
9/9 [==============================] - 15s 2s/step - loss: 17.6378 - mae: 15.6748 - val_loss: 17.1028 - val_mae: 15.1398

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00229: val_loss did not improve from 17.08047

Epoch 00229: val_loss did not improve from 17.08047

Epoch 00229: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00229: saving model to keras_model_test/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 230/300
9/9 [==============================] - 14s

9/9 [==============================] - 15s 2s/step - loss: 17.6270 - mae: 15.6648 - val_loss: 17.0604 - val_mae: 15.0982

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00244: val_loss did not improve from 17.05605

Epoch 00244: val_loss did not improve from 17.05605

Epoch 00244: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00244: saving model to keras_model_test/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 245/300
9/9 [==============================] - 15s 2s/step - loss: 17.7495 - mae: 15.7872 - val_loss: 17.0589 - val_mae: 15.0967

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00245: val_loss did not improve from 17.05605

Epoch 00245: val_loss did not improve from 17.05605

Epoch 00245: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00245: saving model to keras_model_test/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 246/300
9/9 [==============================] - 15s

9/9 [==============================] - 15s 2s/step - loss: 17.6930 - mae: 15.7313 - val_loss: 17.0635 - val_mae: 15.1018

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00260: val_loss did not improve from 17.04886

Epoch 00260: val_loss did not improve from 17.04886

Epoch 00260: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00260: saving model to keras_model_test/KERAS_check_model_last_weights.h5

Epoch 00260: saving model to keras_model_test/KERAS_check_model_epoch260.h5

***callbacks end***

Epoch 261/300
9/9 [==============================] - 15s 2s/step - loss: 17.5793 - mae: 15.6175 - val_loss: 17.0533 - val_mae: 15.0915

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00261: val_loss did not improve from 17.04886

Epoch 00261: val_loss did not improve from 17.04886

Epoch 00261: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00261: saving model to keras_model_test/KERAS_check_model_last_weights.h5



9/9 [==============================] - 15s 2s/step - loss: 17.6844 - mae: 15.7229 - val_loss: 17.0590 - val_mae: 15.0975

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00277: val_loss did not improve from 17.04605

Epoch 00277: val_loss did not improve from 17.04605

Epoch 00277: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00277: saving model to keras_model_test/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 278/300
9/9 [==============================] - 14s 2s/step - loss: 17.5379 - mae: 15.5764 - val_loss: 17.0597 - val_mae: 15.0982

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00278: val_loss did not improve from 17.04605

Epoch 00278: val_loss did not improve from 17.04605

Epoch 00278: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00278: saving model to keras_model_test/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 279/300
9/9 [==============================] - 15s

9/9 [==============================] - 14s 2s/step - loss: 17.5386 - mae: 15.5772 - val_loss: 17.0443 - val_mae: 15.0829

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00294: val_loss improved from 17.04437 to 17.04425, saving model to keras_model_test/KERAS_check_best_model.h5

Epoch 00294: val_loss improved from 17.04437 to 17.04425, saving model to keras_model_test/KERAS_check_best_model_weights.h5

Epoch 00294: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00294: saving model to keras_model_test/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 295/300
9/9 [==============================] - 15s 2s/step - loss: 17.6161 - mae: 15.6548 - val_loss: 17.0454 - val_mae: 15.0841

***callbacks***
saving losses to keras_model_test/losses.log

Epoch 00295: val_loss did not improve from 17.04425

Epoch 00295: val_loss did not improve from 17.04425

Epoch 00295: saving model to keras_model_test/KERAS_check_model_last.h5

Epoch 00295: saving 

In [12]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
y_unp = model.predict(x_test_em_barrel)
print("Score unpruned: {}".format(mean_absolute_error(y_test_targets, y_unp)))

Score unpruned: 15.924873352050781


In [19]:
physical_devices = tf.config.list_physical_devices()
for d in physical_devices:
    print(d)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [20]:
has_tpus = np.any([d.device_type == "TPU" for d in physical_devices])

if has_tpus:
  TPU_WORKER = 'local'

  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
      tpu=TPU_WORKER, job_name='tpu_worker')
  if TPU_WORKER != 'local':
    tf.config.experimental_connect_to_cluster(resolver, protocol='grpc+loas')
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
  print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

  cur_strategy = strategy
else:
  cur_strategy = tf.distribute.get_strategy()

In [22]:
run_config = {
  "output_dir": tempfile.mkdtemp(),
  "goal": goal,
  "quantization_config": quantization_config,
  "learning_rate_optimizer": False,
  "transfer_weights": False,
  "mode": "random",
  "seed": 42,
  "limit": limit,
  "tune_filters": "layer",
  "tune_filters_exceptions": "^dense",
  "distribution_strategy": cur_strategy,
  # first layer is input, layer two layers are softmax and flatten
  "layer_indexes": range(1, len(model.layers) - 1),
  "max_trials": 20
}

print("quantizing layers:", [model.layers[i].name for i in run_config["layer_indexes"]])

quantizing layers: ['up_sampling', 'batch_norm', 'conv', 'batch_norm_1', 'leaky_relu', 'max_pooling2D', 'conv_1', 'batch_norm_2', 'leaky_relu_1', 'conv_2', 'batch_norm_3', 'leaky_relu_2', 'max_pooling2D_1', 'conv_3', 'batch_norm_4', 'leaky_relu_3', 'conv_4', 'batch_norm_5', 'leaky_relu_4', 'max_pooling2D_2', 'conv_5', 'batch_norm_6', 'leaky_relu_5', 'conv_6', 'batch_norm_7', 'leaky_relu_6', 'max_pooling2D_3', 'conv_7', 'batch_norm_8', 'leaky_relu_7', 'conv2d_batchnorm_8', 'leaky_relu_8', 'flatten', 'dense', 'batch_norm_10', 'leaky_relu_9', 'dense_1', 'batch_norm_11', 'leaky_relu_10', 'dense_2']


In [ ]:
custom_objects = {}
autoqk = AutoQKeras(model, metrics=["mae"], custom_objects=custom_objects, **run_config)

autoqk.fit(x_train_em_barrel, y_train_targets, validation_split=0.1, batch_size=1024, epochs=30)

Limit configuration:{"Dense": [8, 8, 4], "Conv2D": [4, 8, 4], "DepthwiseConv2D": [4, 8, 4], "Activation": [4], "BatchNormalization": []}
learning_rate: 1.8310547034161573e-07
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
em_barrel (InputLayer)       [(None, 56, 11, 4)]       0         
_________________________________________________________________
up_sampling (UpSampling2D)   (None, 56, 55, 4)         0         
_________________________________________________________________
batch_norm (QBatchNormalizat (None, 56, 55, 4)         16        
_________________________________________________________________
conv (QConv2D)               (None, 56, 55, 16)        1600      
_________________________________________________________________
batch_norm_1 (QBatchNormaliz (None, 56, 55, 16)        64        
_________________________________________________________________
leaky_relu (LeakyR

learning_rate: 1.8310547034161573e-07
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
em_barrel (InputLayer)       [(None, 56, 11, 4)]       0         
_________________________________________________________________
up_sampling (UpSampling2D)   (None, 56, 55, 4)         0         
_________________________________________________________________
batch_norm (QBatchNormalizat (None, 56, 55, 4)         16        
_________________________________________________________________
conv (QConv2D)               (None, 56, 55, 32)        3200      
_________________________________________________________________
batch_norm_1 (QBatchNormaliz (None, 56, 55, 32)        128       
_________________________________________________________________
leaky_relu (LeakyReLU)       (None, 56, 55, 32)        0         
_________________________________________________________________
max_pooling2D (MaxPooli

9/9 [==============================] - 23s 2s/step - loss: 88.9567 - mae: 81.1934 - trial: 3436884.0000 - score: 1.1230 - val_loss: 90.3903 - val_mae: 82.6279 - val_trial: 3436884.0000 - val_score: 1.1230
Epoch 2/30


In [24]:
qmodel = autoqk.get_best_model()

learning_rate: 1.8310547034161573e-07
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
em_barrel (InputLayer)       [(None, 56, 11, 4)]       0         
_________________________________________________________________
up_sampling (UpSampling2D)   (None, 56, 55, 4)         0         
_________________________________________________________________
batch_norm (QBatchNormalizat (None, 56, 55, 4)         16        
_________________________________________________________________
conv (QConv2D)               (None, 56, 55, 32)        3200      
_________________________________________________________________
batch_norm_1 (QBatchNormaliz (None, 56, 55, 32)        128       
_________________________________________________________________
leaky_relu (LeakyReLU)       (None, 56, 55, 32)        0         
_________________________________________________________________
max_pooling2D (MaxPooli

In [25]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
y_q = qmodel.predict(x_test_em_barrel)
print("Score unpruned: {}".format(mean_absolute_error(y_test_targets, y_q)))

Score unpruned: 78.26563262939453
